In [480]:
# AI20BTECH11025
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from copy import deepcopy

In [481]:
board_size = 3
class Tictactoe():
    def __init__(self, n):
        self.size = n
        self.board = np.zeros((n,n), dtype='U1')
        self.player = 'x'
        self.end = False

    def act(self, pos):
        if self.board[pos] == '' :
            self.board[pos] = self.player
            self.player = 'x' if self.player == 'o' else 'o'
        else : 
            print('Move is invalid')
     
    def print_board(self):
        print(self.board[0,0], ' | ',self.board[0,1], ' | ',self.board[0,2])
        print('-----------')
        print(self.board[1,0], ' | ',self.board[1,1], ' | ',self.board[1,2])
        print('------------')
        print(self.board[2,0], ' | ',self.board[2,1], ' | ',self.board[2,2])        
    
    def state(self):
        return str(self.board.reshape(self.size * self.size))

    def availablepositions(self):
        positions = []
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i, j] == '': 
                    positions.append((i, j))
        return positions

    def checkwinner(self):
        # Along any row
        for i in range(self.size):
            if sum(np.char.count(self.board[i, :], 'x')) == self.size:
                self.end = True
                return 'x'
            if sum(np.char.count(self.board[i, :], 'o')) == self.size:
                self.end = True
                return 'o'
        
        # Along any column
        for i in range(self.size):
            if sum(np.char.count(self.board[:, i], 'x')) == self.size:
                self.end = True
                return 'x'
            if sum(np.char.count(self.board[:, i], 'o')) == self.size:
                self.end = True
                return 'o'

        # Along any diagonal
        if sum(np.char.count(self.board.diagonal(), 'x')) == self.size:
              self.end = True
              return 'x'
        if sum(np.char.count(self.board.diagonal(), 'o')) == self.size:
              self.end = True
              return 'o'
        if sum(np.char.count((self.board[::-1]).diagonal(), 'x')) == self.size:
              self.end = True
              return 'x'
        if sum(np.char.count((self.board[::-1]).diagonal(), 'o')) == self.size:
              self.end = True
              return 'o'
            
        # Tie : No available positions
        if len(self.availablepositions()) == 0: 
              self.end = True
              return 0

        # If any moves available
        self.end = False
        return -1

    def reward(self):
        if self.checkwinner() == 'x':
            return 1
        if self.checkwinner() == 'o':
            return -1
        if self.checkwinner() == 0:
            return 0.5
        return 0

In [482]:
# Defining a tree node
class Node(Tictactoe):
    def __init__(self, player, parent, board):
        super().__init__(board_size)
        self.visits = 0
        self.score = 0
        self.player = player
        self.board = board
        self.parent = parent
        self.children = {}   # Dictionary of child nodes
        self.is_leaf = True 

        # Check if the node is a leaf node
        # Any moves available -> Not a leaf node
        if self.checkwinner() == -1:
            self.is_leaf = False

        # If leaf, then the node is fully expanded
        self.fully_expanded = self.is_leaf

In [483]:
class MCTS():
    def __init__(self, rootnode, K):
        self.rootnode = rootnode
        self.K = K

    def mcts(self):
        for k in range(self.K):
            leaf = self.traverse(self.rootnode)

            temp_leaf = deepcopy(leaf)
            simresult = self.rollout(temp_leaf)

            self.backpropogation(simresult, leaf)

        best_score = float('-inf')
        for child in self.rootnode.children.values():
            ucb = (child.score / child.visits) 
            if ucb > best_score:
                best_score = ucb
                best_child = [(i, j) for i in range(board_size) for j in range(board_size) if child.board[i, j] != self.rootnode.board[i, j]]

        return best_child[0]

    def backpropogation(self, simresult, node):
        while node is not None:
            node.score += simresult
            node.visits += 1
            node = node.parent
            
    def rollout(self, node):
        while node.checkwinner() == -1:
            random_move = random.choice(node.availablepositions())
            node.act(random_move)
        
        return node.reward()
        
    def traverse(self, node):
        while node.checkwinner() == -1:
            if node.fully_expanded:
                node = self.next_best_move(node)
            else:
                return self.randomexpandedchild(node)         
        
        return node

    def randomexpandedchild(self, node):
        pos = node.availablepositions()
        for p in pos:
            if str(p) not in node.children:
                new_node = Node(node.player, node, node.board.copy())
                new_node.act(p)

                if new_node.checkwinner() == -1:
                    new_node.is_leaf = False
                else:
                    new_node.is_leaf = True

                node.children[str(p)] = new_node

                if len(pos) == len(node.children):
                    node.fully_expanded = True       

                return new_node
                
    def next_best_move(self, node):
        best_score = float('-inf')
        best_child = []
        for child in node.children.values():
            if child.visits == 0:
                return child
            ucb = (child.score / child.visits) + np.sqrt(node.visits / child.visits)
            if ucb > best_score:
                best_score = ucb
                best_child.append(child)
       
        return random.choice(best_child)

In [484]:
class RandomAgent():

    def __init__(self):
        pass
    
    def policy(self, env):
        positions = env.availablepositions()
        return random.choice(positions)

In [485]:
class SafeAgent():

    def __init__(self):
        pass
    
    def policy(self, env):
        positions = env.availablepositions()
        
        # Win along any row
        for i in range(env.size):
            if sum(np.char.count(env.board[i, :], 'x')) == env.size - 1 and sum(np.char.count(env.board[i, :], 'o')) == 0:
                for j in range(env.size):
                    if env.board[i, j] == '': 
                        return (i, j)

        # Win along any column
        for i in range(env.size):
            if sum(np.char.count(env.board[:, i], 'x')) == env.size - 1 and sum(np.char.count(env.board[:, i], 'o')) == 0:
                for j in range(env.size):
                    if env.board[j, i] == '': 
                        return (j, i)

        # Win along any diagonal
        if sum(np.char.count(env.board.diagonal(), 'x')) == env.size - 1 and sum(np.char.count(env.board.diagonal(), 'o')) == 0:
            for i in range(env.size):
                if env.board[i, i] == '':
                    return (i, i)

        if sum(np.char.count((env.board[::-1]).diagonal(), 'x')) == env.size - 1 and sum(np.char.count((env.board[::-1]).diagonal(), 'o')) == 0:
            for i in range(env.size):
                if env.board[i, env.size - 1 - i] == '':
                    return (i, env.size - 1 - i)


        # Block along any row
        for i in range(env.size):
            if sum(np.char.count(env.board[i, :], 'o')) == env.size - 1 and sum(np.char.count(env.board[i, :], 'x')) == 0:
                for j in range(env.size):
                    if env.board[i, j] == '': 
                        return (i, j)

        # Block along any column
        for i in range(env.size):
            if sum(np.char.count(env.board[:, i], 'o')) == env.size - 1 and sum(np.char.count(env.board[:, i], 'x')) == 0:
                for j in range(env.size):
                    if env.board[j, i] == '': 
                        return (j, i)

        # Block along any diagonal
        if sum(np.char.count(env.board.diagonal(), 'o')) == env.size - 1 and sum(np.char.count(env.board.diagonal(), 'x')) == 0:
            for i in range(env.size):
                if env.board[i, i] == '':
                    return (i, i)

        if sum(np.char.count((env.board[::-1]).diagonal(), 'o')) == env.size - 1 and sum(np.char.count((env.board[::-1]).diagonal(), 'x')) == 0:
            for i in range(env.size):
                if env.board[i, env.size - 1 - i] == '':
                    return (i, env.size - 1 - i)

        # No available safe moves
        # Pick a random action
        positions = env.availablepositions()
        return random.choice(positions)

In [486]:
class MCTSAgent():
    def __init__(self, K):
        self.K = K
    def policy(self, env):
        self.player = env.player
        self.board = env.board.copy()
        if env.player == 'o':
            for i in range(board_size):
                for j in range(board_size):
                    if self.board[i, j] == 'x':
                        self.board[i, j] = 'p'
                    if self.board[i, j] == 'o':
                        self.board[i, j] = 'q'
            for i in range(board_size):
                for j in range(board_size):
                    if self.board[i, j] == 'p':
                        self.board[i, j] = 'o'
                    if self.board[i, j] == 'q':
                        self.board[i, j] = 'x'
        n = Node('x', None, self.board)
        m = MCTS(n, self.K)
        (i, j) = m.mcts()
        return (i, j)

# Testing the efficacy of the MCTS agent 

A suitable board configuration in which the MCTS agent is one move away from win

In [487]:
ex_board = np.array([['x','o','o'], ['','x',''], ['','','']])
print('Given board position')
print(ex_board)
n = Node('x', None, ex_board.copy())
m = MCTS(n, 1000)
print('Move by MCTS')
(i, j) = m.mcts()
print('(',i,',' ,j,')')
print('Final board position')
ex_board[i, j] = 'x'
print(ex_board)

Given board position
[['x' 'o' 'o']
 ['' 'x' '']
 ['' '' '']]
Move by MCTS
( 2 , 2 )
Final board position
[['x' 'o' 'o']
 ['' 'x' '']
 ['' '' 'x']]


A suitable board configuration in which the MCTS agent is one move away from loss

In [488]:
ex_board = np.array([['o','x',''], ['x','o',''], ['o','','']])
print('Given board position')
print(ex_board)
n = Node('x', None, ex_board.copy())
m = MCTS(n, 1000)
print('Move by MCTS')
(i, j) = m.mcts()
print('(',i,',' ,j,')')
print('Final board position')
ex_board[i, j] = 'x'
print(ex_board)

Given board position
[['o' 'x' '']
 ['x' 'o' '']
 ['o' '' '']]
Move by MCTS
( 2 , 2 )
Final board position
[['o' 'x' '']
 ['x' 'o' '']
 ['o' '' 'x']]


The board configuration where the opponent has made the first move and has occupied
the centre square

In [489]:
ex_board = np.array([['','',''], ['','o',''], ['','','']])
print('Given board position')
print(ex_board)
n = Node('x', None, ex_board.copy())
m = MCTS(n, 500)
print('Move by MCTS')
(i, j) = m.mcts()
print('(',i,',' ,j,')')
print('Final board position')
ex_board[i, j] = 'x'
print(ex_board)

Given board position
[['' '' '']
 ['' 'o' '']
 ['' '' '']]
Move by MCTS
( 2 , 2 )
Final board position
[['' '' '']
 ['' 'o' '']
 ['' '' 'x']]


# Play against Random Agent

In [490]:
def test(agent, opponent, num_games):
    num_wins = 0
    num_draws = 0
    for i in range(num_games):
        env = Tictactoe(board_size)

        while(env.end == False):
            env.act(agent.policy(env))

            winner = env.checkwinner()
            if winner == -1:
                env.act(opponent.policy(env))
                winner = env.checkwinner()
        
        if (env.checkwinner() == 'x'): num_wins += 1
        if (env.checkwinner() == 0): num_draws += 1

    return num_wins, num_draws

In [491]:
K = 100

# Testing with Random Agent
random_agent_test = RandomAgent()
agent = MCTSAgent(K)
win_count = 0
draw_count = 0
for i in range(1000):
    x, y = test(agent, random_agent_test, 1)
    win_count += x
    draw_count += y

print('Wins against Random agent: ', win_count)
print('Draw against Random agent: ', draw_count)
print('Loss against Random agent: ', 1000 - win_count - draw_count)

Wins against Random agent:  968
Draw against Random agent:  26
Loss against Random agent:  6


In [492]:
K = 100

# Testing with Safe Agent
random_agent_test = SafeAgent()
agent = MCTSAgent(K)
win_count = 0
draw_count = 0
for i in range(1000):
    x, y = test(agent, random_agent_test, 1)
    win_count += x
    draw_count += y

print('Wins against Safe agent: ', win_count)
print('Draw against Safe agent: ', draw_count)
print('Loss against Safe agent: ', 1000 - win_count - draw_count)

Wins against Safe agent:  460
Draw against Safe agent:  521
Loss against Safe agent:  19


In [494]:
K = 100

# Testing with MCTS Agent
random_agent_test = MCTSAgent(K)
agent = MCTSAgent(K)
win_count = 0
draw_count = 0
for i in range(1000):
    x, y = test(agent, random_agent_test, 1)
    win_count += x
    draw_count += y

print('Wins against MCTS agent: ', win_count)
print('Draw against MCTS agent: ', draw_count)
print('Loss against MCTS agent: ', 1000 - win_count - draw_count)

Wins against MCTS agent:  585
Draw against MCTS agent:  351
Loss against MCTS agent:  64
